In [2]:
import numpy as np
import cvxpy as cp
from tabulate import tabulate

BitString = list[int]

# Set the possible values of each bit: {0, 1}
a_range = b_range = x_range = y_range = range(2)


def generate_bitstrings(n: int) -> list[BitString]:
    """
    Function that generates all possible bitstrings of length n.
    """
    if n == 0:
        return [[]]
    else:
        previous_bitstrings = generate_bitstrings(n - 1)
        current_bitstrings = []
        for bitstring in previous_bitstrings:
            current_bitstrings.append(bitstring + [0])
            current_bitstrings.append(bitstring + [1])
        return current_bitstrings

TypeError: 'type' object is not subscriptable

Here, the bitstring lengths that Alice and Bob receive can be set. $x=x\_range^n$ is for Alice, $y=y\_range^m$ is for Bob.

In [ ]:
# These variables may be changed
m = 3
n = 3

# These variables shouldn't be changed
# Generates all possible bitstrings for x and y
x_values = generate_bitstrings(m)
y_values = generate_bitstrings(n)

$q(x,y)$ is the distribution function, which returns the probability of Alice receiving $x$ and Bob receiving $y$. For a uniform distribution, `q_uniform` can be called.

In [ ]:
def q_uniform():
    x_possibilities = len(x_values)
    y_possibilities = len(y_values)

    return 1 / (x_possibilities * y_possibilities)


# This function may be changed
def q(x: BitString, y: BitString):
    return q_uniform()

Alice and Bob win the game when $a \oplus b = f(x,y)$.
$f(x,y)$ can be set here.

In [ ]:
# This function may be changed
def f(x: BitString, y: BitString):
    return  (y[0] * x[0]) ^ (y[1] * x[1]) ^ (y[2] * x[2])


# This function shouldn't be changed
def V(a: int, b: int, x: BitString, y: BitString):
    return a ^ b == f(x, y)

We want to optimize the following problem

\begin{equation}
    \omega(G) = \max \sum_{xy} q(x,y) \sum_{ab} V(a,b\,|\,x,y) \cdot p(a, b\,|\,x, y)
\end{equation}

We are going to brute-force the best strategy, so we need to loop over all possible values of $p(a, b|x, y)$.

Note that we need to loop over $2^{2^m}$ strategies for Alice and $2^{2^n}$ strategies for Bob, so combined that is $2^{2^m + 2^n}$ strategies. For each strategy, we need to check every possible combination of $x, y$ ($2^m * 2^n$ combinations) and check if Alice and Bob win according to the function $V(a, b|x, y)$.

In total, we need to check if a combination of $x, y, a, b$ wins $2^{2^m+2^n+m+n}$ times.

In [ ]:
def create_strategies_shape(bit_possibilities, bitstring_length):
    shape = [-1]
    for _ in range(bitstring_length):
        shape.append(bit_possibilities)

    return shape


# Create possible strategies
strategies_A = np.array(generate_bitstrings(len(x_range) ** m)).reshape(*create_strategies_shape(len(x_range), m))
strategies_B = np.array(generate_bitstrings(len(y_range) ** n)).reshape(*create_strategies_shape(len(y_range), n))

# Keep track of the best strategy up till now
best_strategy = None
best_value = 0

for strategy_A in strategies_A:
    for strategy_B in strategies_B:
        # Calculate the winning probability for this strategy, according to the formula above.
        sum = 0
        for x in x_values:
            for y in y_values:
                prob = q(x, y)
                a = strategy_A[(*x,)]
                b = strategy_B[(*y,)]
                sum += prob * V(a, b, x, y)

        if sum > best_value:
            best_value = sum
            best_strategy = {'A': strategy_A, 'B': strategy_B}

print('Maximal winning probability:', best_value)
print('Best strategy:')
print('Alice:', best_strategy['A'])
print('Bob:', best_strategy['B'])